In [1]:
import numpy as np
try:
    import ruamel_yaml as yaml
except ImportError:
    from ruamel import yaml
from email.utils import formatdate
BlockMap = yaml.comments.CommentedMap

In [2]:
def FlowMap(*args, **kwargs):
    m = yaml.comments.CommentedMap(*args, **kwargs)
    m.fa.set_flow_style()
    return m

def FlowList(*args, **kwargs):
    lst = yaml.comments.CommentedSeq(*args, **kwargs)
    lst.fa.set_flow_style()
    return lst

In [3]:
emitter = yaml.YAML()
emitter.width = 70
out_name='test.yaml'

In [4]:
TChemSimulation = {'title':'Zero D ignition simulation',\
                  'date':formatdate(localtime=True)}

In [5]:
FileNames = {'chem':'chem.inp', 'thermo':'term.dat'}

In [6]:
TimeIntegration = {'tbegin':0,'tend':1,\
                   'dtmin':1e-10,'dtmax':1e-2,\
                   'absolute_tolerance_newton':1e-16,'relative_tolerance_newton':1e-8,\
                   'relative_tolerance_time':1e-6,'absolute_tolerance_time':1e-12,\
                   'max_newton_iterations':20, 'max_time_iterations':100
                   }

In [7]:
OutputFile = {'file_name':'IgnSolution.dat', 'output_frequency':10}

In [8]:
IgnitionDelayTime = {'threshold_temperature':1500,'compute_only_ignition_delay_time':'false',\
                     'file_name_second_derivative':'IgnitionDelayTime.dat',\
                     'file_name_Tthreshold':"IgnitionDelayTimeTthreshold.dat"}

In [9]:
SimulationSettings = {'verbose':'false','team_size':-1,'vector_size':-1}

In [10]:
Header = ['T','P','H2','O2']
sample = np.array([[900, 101325, 0.12, 0.88],[800, 101325, 0.2, 0.8]])

In [11]:
DesignOfExperiment = {'variables':FlowList(Header)}
Nsamples = 2
for i in range(Nsamples):
    sampleNo = sample[i,:].tolist()
    DesignOfExperiment.update({'condition_No_'+str(i):FlowList(sampleNo)})

In [12]:
ReactionIndex = [1, 2, 3 ]
Nreaction = len(ReactionIndex)
μ=0
σ=0.25
A = np.random.lognormal(μ, σ, size=(Nsamples, Nreaction))
β = np.ones((Nsamples, Nreaction))
E = np.random.lognormal(μ, σ, size=(Nsamples, Nreaction))

In [13]:
DesignOfExperiment.update({'reaction_index':FlowList(ReactionIndex)})
for i in range(Nsamples):
    DesignOfExperiment.update({'modifier_pre_exporencial_No_'+str(i):FlowList( A[i,:].tolist() )})
    DesignOfExperiment.update({'modifier_temperature_coefficient_No_'+str(i):FlowList( β[i,:].tolist() )})
    DesignOfExperiment.update({'modifier_activation_energy_No_'+str(i):FlowList( E[i,:].tolist() )})

In [14]:
with open(out_name, 'w') as dest:
    emitter.dump({'TChem Simulation':TChemSimulation}, dest)
    emitter.dump({'FileNames':FileNames}, dest)
    emitter.dump({'TimeIntegration':TimeIntegration}, dest)
    emitter.dump({'OutputFile':OutputFile}, dest)
    emitter.dump({'IgnitionDelayTime':IgnitionDelayTime}, dest)
    emitter.dump({'SimulationSettings':SimulationSettings}, dest)
    emitter.dump({'DesignOfExperiment':DesignOfExperiment}, dest)  